In [2]:
from pyffmpeg import FFmpeg
import os
ff = FFmpeg()

2023-03-06 21:57:28,836 - pyffmpeg.FFmpeg - INFO - FFmpeg Initialising
2023-03-06 21:57:28,837 - pyffmpeg.FFmpeg - INFO - Save directory: .
2023-03-06 21:57:28,838 - pyffmpeg.misc.Paths - INFO - bin folder: /Users/vedant/.pyffmpeg/bin
2023-03-06 21:57:28,839 - pyffmpeg.misc.Paths - INFO - Inside load_ffmpeg_bin
2023-03-06 21:57:28,840 - pyffmpeg.FFmpeg - INFO - FFmpeg file: /Users/vedant/.pyffmpeg/bin/ffmpeg


In [ ]:
# ff.options("-f lavfi -i color=c=black:s=1280x720:r=5 -i ../resources/Nightmare.wav -crf 0 -c:a copy -pix_fmt yuv420p -shortest output.mp4")

In [ ]:
# t=f"-hwaccel videotoolbox -f lavfi -i color=c=black:s=1280x720:r=25/1 -i {os.path.join(os.getcwd(),'../resources/Nightmare.wav')} -c:v h264_videotoolbox -q:v 50 -c:a aac -shortest ../resources/output.mp4"
# print(t)
# ff.options(t)

In [3]:
# WORKING METHODS

# ff.options("-t 60 -s 1920x1080 -f rawvideo -pix_fmt rgb24 -r 25 -i /dev/zero empty.mkv")
# ff.options("-t 60 -f lavfi -i color=c=black:s=1920x1080 -c:v libx264 -tune stillimage -pix_fmt yuv420p output.mp4")
ff.options("-f lavfi -i color=c=black:s=1920x1080 -i ../resources/Nightmare.wav -c:v libx264 -tune stillimage -pix_fmt yuv420p -shortest -c:a aac -b:a 128k Nightmare.mp4")

2023-03-06 21:57:31,431 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 21:57:31,432 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 21:57:31,433 - pyffmpeg.FFmpeg - INFO - Shell: True


True

# Lyrics Transcription

In [54]:
from datetime import timedelta
import os
import whisper

def transcribe_audio(path, model):
    file_name = os.path.basename(path)[:-4]
    model = whisper.load_model(model) # Change this to your desired model
    print("Whisper model loaded.")
    transcribe = model.transcribe(audio=path)
    segments = transcribe['segments']

    for segment in segments:
        startTime = str(0)+str(timedelta(seconds=int(segment['start'])))+',000'
        endTime = str(0)+str(timedelta(seconds=int(segment['end'])))+',000'
        text = segment['text']
        segmentId = segment['id']+1
        segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] == ' ' else text}\n\n"

        srtFilename = os.path.join(os.getcwd(), f"{file_name}.srt")
        # if not os.path.exists(os.path.dirname(srtFilename)):
        #     os.makedirs(os.path.dirname(srtFilename))
        with open(srtFilename, 'a', encoding='utf-8') as srtFile:
            srtFile.write(segment)

    return srtFilename

In [8]:
transcribe_audio("../resources/Nightmare.wav")

Whisper model loaded.


/Users/vedant/miniconda3/envs/scalable-asset-generation/lib/python3.10/site-packages/whisper/transcribe.py:113: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


'/Users/vedant/Desktop/Programming/WMG/scalable-asset-generation-1/src/SrtFiles/VIDEO_FILENAME.srt'

# Add lyrics to the video

In [12]:
subtitle_path=transcribe_audio("../resources/Nightmare.wav")
ff.options("""-f lavfi -i color=c=black:s=1920x1080 -i ../resources/Nightmare.wav -c:v libx264 -tune stillimage -pix_fmt yuv420p -shortest -c:a aac -b:a 128k Nightmare.mp4""")


Whisper model loaded.


/Users/vedant/miniconda3/envs/scalable-asset-generation/lib/python3.10/site-packages/whisper/transcribe.py:113: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
2023-03-06 22:09:38,175 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 22:09:38,176 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 22:09:38,177 - pyffmpeg.FFmpeg - INFO - Shell: True
2023-03-06 22:10:05,150 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 22:10:05,152 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 22:10:05,153 - pyffmpeg.FFmpeg - INFO - Shell: True


True

In [15]:
subtitle_path

'/Users/vedant/Desktop/Programming/WMG/scalable-asset-generation-1/src/SrtFiles/VIDEO_FILENAME.srt'

In [19]:
# ff.options(f"""Nightmare.mp4 -vf subtitles=SrtFiles/VIDEO_FILENAME.srt Nightmare_subs.mp4 """)

2023-03-06 22:14:38,362 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 22:14:38,364 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 22:14:38,365 - pyffmpeg.FFmpeg - INFO - Shell: True


True

In [23]:
# ff.options("-i Nightmare.mp4 -i SrtFiles/VIDEO_FILENAME.srt -c copy -c:s mov_text outfile.mp4")

2023-03-06 22:17:21,376 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 22:17:21,378 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 22:17:21,378 - pyffmpeg.FFmpeg - INFO - Shell: True


True

In [38]:
ff.options(f"-i {subtitle_path} subtitles_1.ass")

2023-03-06 22:35:23,039 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 22:35:23,041 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 22:35:23,041 - pyffmpeg.FFmpeg - INFO - Shell: True


True

In [34]:
ff.options("-i SrtFiles/VIDEO_FILENAME.srt subtitles.ass")
ff.options("""-i Nightmare.mp4 -vf "subtitles=subtitles.ass:force_style='Alignment=10,Fontsize=48'" -crf 1 -c:a copy mysubtitledmovie.mp4""")

2023-03-06 22:29:19,941 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 22:29:19,943 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 22:29:19,944 - pyffmpeg.FFmpeg - INFO - Shell: True
2023-03-06 22:29:19,991 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 22:29:19,992 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 22:29:19,993 - pyffmpeg.FFmpeg - INFO - Shell: True


True

# Combined processing

In [52]:
# def make_lyric_video(path_to_audio_file:str):
    # file_name = os.path.basename(path_to_audio_file)[:-4]
    # subtitle_path=transcribe_audio(path_to_audio_file)
    # ff.options(f"""-f lavfi -i color=c=black:s=1920x1080 -i {path_to_audio_file} -c:v libx264 -tune stillimage -pix_fmt yuv420p -shortest -c:a aac -b:a 128k {file_name}.mp4""")
    # ff.options(f"-i {subtitle_path} {file_name}.ass")
    # ff.options(f"""-i {file_name}.mp4 -vf "subtitles={file_name}.ass:force_style='Alignment=10,Fontsize=48'" -crf 1 -c:a copy {file_name}.mp4""")

In [37]:
os.path.basename('../resources/Nightmare.wav')[:-4]

'Nightmare'

In [59]:
# make_lyric_video('../resources/TakeOnMe.wav')
path_to_audio_file='../resources/TakeOnMe.wav'

from dataclasses import dataclass
import numpy as np
import librosa
import subprocess as sp
from shutil import rmtree
import soundfile as sf

@dataclass
class DeMixedAudio4():
    sample_rate: int
    bass : np.ndarray
    drums : np.ndarray
    other : np.ndarray
    vocals : np.ndarray

@dataclass
class DeMixedAudio6():
    sample_rate: int
    bass : np.ndarray
    drums : np.ndarray
    other : np.ndarray
    vocals : np.ndarray
    piano : np.ndarray
    guitar : np.ndarray

def get_stems(path_to_file,model='htdemucs',output_path='../resources/tmp',delete_temp_files=True):
    file_name = os.path.basename(path_to_file)
    sp.run(["python3", "-m", "demucs.separate", "-o", output_path, "-n", model, path_to_file])
    # print("Demucs done, loading files")
    stems={}
    if model in ['htdemucs','htdemucs_ft']:
        for stem in ['bass','drums','other','vocals']:
            x,sr = librosa.load(os.path.join(output_path, model,file_name[:file_name.find('.')],f"{stem}.wav"), sr=None)
            stems[stem] = x
        if delete_temp_files:
            print("Deleting temp files")
            rmtree(f"{output_path}/{model}/{file_name[:file_name.find('.')]}")
        return DeMixedAudio4(sr, bass=stems['bass'], drums=stems['drums'], other=stems['other'], vocals=stems['vocals'])#stems
    elif model == 'htdemucs_6s':
        for stem in ['bass','drums','other','vocals','piano', 'guitar']:
            x,sr = librosa.load(os.path.join(output_path, model,file_name[:file_name.find('.')],f"{stem}.wav"), sr=None)
            stems[stem] = x
        if delete_temp_files:
            print("Deleting temp files")
            rmtree(f"{output_path}/{model}/{file_name[:file_name.find('.')]}")
        return DeMixedAudio6(sr, bass=stems['bass'], drums=stems['drums'], other=stems['other'], vocals=stems['vocals'], piano=stems['piano'], guitar=stems['guitar'])#stems
    

def make_lyric_video(path_to_audio_file:str, whisper_model:str='small.en', run_on_stems:bool=True, delete_tmp_files:bool=True):
    file_name = os.path.basename(path_to_audio_file)[:-4]
    if run_on_stems:
        if not os.path.exists(os.path.join(os.getcwd(),f'../resources/tmp/htdemucs_ft/{file_name}')):
            stems = get_stems(path_to_audio_file, model='htdemucs_ft', delete_temp_files=delete_tmp_files)
            sf.write(f"{file_name}.wav", stems.vocals, stems.sample_rate)
        else:
            x,sr=librosa.load(os.path.join(os.getcwd(),f'../resources/tmp/htdemucs_ft/{file_name}'),sr=None)
            sf.write(f"{file_name}.wav", x, sr)
        subtitle_path=transcribe_audio(f"{file_name}.wav", whisper_model)
    else:
        subtitle_path=transcribe_audio(path_to_audio_file, whisper_model)
    ff.options(f"""-f lavfi -i color=c=black:s=1920x1080 -i {path_to_audio_file} -c:v libx264 -tune stillimage -pix_fmt yuv420p -shortest -c:a aac -b:a 128k {file_name}_tmp.mp4""")
    ff.options(f"-i {subtitle_path} {file_name}.ass")
    ff.options(f"""-i {file_name}_tmp.mp4 -vf "subtitles={file_name}.ass:force_style='Alignment=10,Fontsize=36'" -crf 1 -c:a copy {file_name}.mp4""")

In [60]:
make_lyric_video('../resources/TakeOnMe.wav', whisper_model='base.en', run_on_stems=True, delete_tmp_files=False)

Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /Users/vedant/Desktop/Programming/WMG/scalable-asset-generation-1/resources/tmp/htdemucs_ft
Separating track ../resources/TakeOnMe.wav


100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [03:21<00:00,  1.22seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [03:30<00:00,  1.17seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [03:25<00:00,  1.20seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [03:19<00:00,  1.23seconds/s]


Whisper model loaded.


/Users/vedant/miniconda3/envs/scalable-asset-generation/lib/python3.10/site-packages/whisper/transcribe.py:113: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
2023-03-06 23:54:55,680 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 23:54:55,683 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 23:54:55,684 - pyffmpeg.FFmpeg - INFO - Shell: True
2023-03-06 23:55:12,608 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 23:55:12,609 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 23:55:12,610 - pyffmpeg.FFmpeg - INFO - Shell: True
2023-03-06 23:55:12,643 - pyffmpeg.FFmpeg - INFO - inside options
2023-03-06 23:55:12,645 - pyffmpeg.FFmpeg - INFO - Options is a String
2023-03-06 23:55:12,646 - pyffmpeg.FFmpeg - INFO - Shell: True
